Inladen data, converteren naar gpkg (dan kun je met sqlite werken en lagen toevoegen)
shp gehaald van https://maps.rijkswaterstaat.nl/dataregister/srv/dut/catalog.search#/metadata/db60a314-5583-437d-a2ff-1e59cc57704f

In [78]:
import os
import logging
from osgeo import ogr, gdal

class Logger:
    def __init__(self, log_dir="log"):
        if not os.path.exists(log_dir):
            os.makedirs(log_dir)
        log_file = os.path.join(log_dir, "conversion.log")
        logging.basicConfig(
            level=logging.INFO,
            format="%(asctime)s - %(levelname)s - %(message)s",
            handlers=[
                logging.FileHandler(log_file),
                logging.StreamHandler()
            ]
        )
        self.logger = logging.getLogger()

    def info(self, message):
        self.logger.info(message)

    def error(self, message):
        self.logger.error(message)

class ShapefileConverter:
    def __init__(self, logger):
        self.logger = logger

    def convert_shapefiles_to_geopackage(self, input_shapefiles, output_geopackage):
        try:
            if not input_shapefiles:
                raise ValueError("No input shapefiles provided.")

            for input_shapefile in input_shapefiles:
                if not os.path.exists(input_shapefile):
                    self.logger.error(f"Input shapefile not found: {input_shapefile}")
                    raise FileNotFoundError(f"Shapefile not found: {input_shapefile}")

                self.logger.info(f"Processing shapefile: {input_shapefile}")

                try:
                    srcDS = gdal.OpenEx(input_shapefile, allowed_drivers=['ESRI Shapefile'])
                    if srcDS is None:
                        raise RuntimeError(f"Failed to open shapefile: {input_shapefile}")

                    options = gdal.VectorTranslateOptions(
                        format='GPKG', accessMode='append', srcSRS='EPSG:25831', dstSRS='EPSG:28992',
                        addFields=True
                    )

                    gdal.VectorTranslate(srcDS=srcDS, destNameOrDestDS=output_geopackage, options=options)
                    self.logger.info(f"Successfully converted: {input_shapefile} to {output_geopackage}")

                    srcDS.Close()

                except Exception as e:
                    self.logger.error(f"Error processing shapefile {input_shapefile}: {e}")
                    raise

        except Exception as e:
            self.logger.error(f"Unexpected error during conversion: {e}")
            raise

        self.logger.info("All shapefiles processed.")
        return 'success'

if __name__ == "__main__":
    # File paths
    input_shapefiles = [
        r"..\data\vaarweg_markering_drijvend_detail\vaarweg_markering_drijvend_detailPoint.shp",
        r"..\data\vaarweg_markering_vast_detail\vaarweg_markering_vast_detailPoint.shp"
    ]
    output_geopackage = r"..\data\converted_datav2.gpkg"

    # Initialize logger and converter
    logger = Logger()
    converter = ShapefileConverter(logger)

    # Convert shapefiles to GeoPackage
    converter.convert_shapefiles_to_geopackage(input_shapefiles, output_geopackage)


2024-12-12 12:29:23,375 - INFO - Processing shapefile: ..\data\vaarweg_markering_drijvend_detail\vaarweg_markering_drijvend_detailPoint.shp
2024-12-12 12:29:23,621 - INFO - Successfully converted: ..\data\vaarweg_markering_drijvend_detail\vaarweg_markering_drijvend_detailPoint.shp to ..\data\converted_datav2.gpkg
2024-12-12 12:29:23,621 - INFO - Processing shapefile: ..\data\vaarweg_markering_vast_detail\vaarweg_markering_vast_detailPoint.shp
2024-12-12 12:29:24,065 - INFO - Successfully converted: ..\data\vaarweg_markering_vast_detail\vaarweg_markering_vast_detailPoint.shp to ..\data\converted_datav2.gpkg
2024-12-12 12:29:24,065 - INFO - All shapefiles processed.


In [ ]:
# Import necessary libraries
import pyarrow as pa
import time
import numpy as np
import geopandas as gp
import pandas as pd
from geopandas.array import from_wkb
from osgeo import ogr

# Enable OGR exceptions for better error handling
ogr.UseExceptions()

# Set pandas display option
pd.set_option('display.max_columns', None)

def load(gpkg_path, layername):
    """
    Loads a specified layer from a GeoPackage and returns it as a GeoDataFrame.

    Parameters:
    - gpkg_path (str): Path to the GeoPackage file.
    - layername (str): Name of the layer to load.

    Returns:
    - gdf (GeoDataFrame): The loaded layer as a GeoDataFrame.
    """
    t1 = time.time()
    gdf = None
    df = None
    datasource = None
    try:
        # Connect to the GeoPackage datasource
        driver = ogr.GetDriverByName('GPKG')
        datasource = driver.Open(gpkg_path, 0)  # 0 = read-only
        if datasource is None:
            raise ValueError(f"Failed to open GeoPackage: {gpkg_path}")

        # Access the specified layer
        layer = datasource.GetLayerByName(layername)
        if layer is None:
            raise ValueError(f"Layer '{layername}' not found in {gpkg_path}")
        logger.info(f"Loaded layer '{layername}' from {gpkg_path}")

        # Create Arrow streams
        arrow_stream = layer.GetArrowStreamAsPyArrow()
        if arrow_stream is None:
            raise RuntimeError("Failed to retrieve Arrow stream from the layer.")
        logger.info("Loaded OGR Arrow stream")

        schema = arrow_stream.schema  # Retrieve schema
        logger.info(f"Schema retrieved: {schema}")

        # Extract field names from the schema
        fields = [field for field in schema]
        field_names = [field.name for field in fields if field.name]  # Ensure field has a name
        logger.info(f"Field names extracted: {field_names}")

        # Define schema_object using field names
        schema_object = pa.schema([schema.field(i) for i in range(schema.num_fields)])
        logger.info(f"Schema object defined: {schema_object}")

        # Create table from batches
        batches = []
        for record_batch in arrow_stream:
            # Access fields by name
            arrays = [record_batch.field(field_name) for field_name in field_names]
            batch = pa.RecordBatch.from_arrays(arrays, schema=schema_object)
            batches.append(batch)

        # Combine batches into a single table
        table = pa.Table.from_batches(batches).combine_chunks()
        logger.info(f"Created PyArrow table with {table.num_rows} rows and {table.num_columns} columns.")

        # Convert table to pandas DataFrame
        df = table.to_pandas()
        logger.info("Converted PyArrow table to pandas DataFrame")

        # Identify geometry column (assuming it's named 'geom' or similar)
        # Adjust the geometry column name as per your dataset
        geometry_column = None
        for col in df.columns:
            if 'geom' in col.lower() or 'geometry' in col.lower():
                geometry_column = col
                break

        if geometry_column is None:
            raise ValueError("No geometry column found in the data.")

        # Convert WKB to geometry using GeoPandas
        geometry = from_wkb(df[geometry_column])
        gdf = gp.GeoDataFrame(df, geometry=geometry, crs = "EPSG:28992")
        gdf = gdf.drop(columns=["geom"]) #mistake somewhere
        logger.info("Converted DataFrame to GeoDataFrame")

        t2 = time.time()
        logger.info(f"Data loaded successfully in {t2 - t1:.2f} seconds.")

        return gdf

    except Exception as e:
        logger.error(f"An error occurred while loading the layer '{layername}': {e}")
        raise

    finally:
        if datasource:
            datasource.Close()
            logger.info(f"Closed the GeoPackage datasource: {gpkg_path}")

# Example Usage
if __name__ == "__main__":

    gpkg_path = r'..\data\converted_datav2.gpkg'  # Update with your actual path
    layer_name = 'vaarweg_markering_drijvend_detailPoint'  # Replace with your actual layer name

    gdf = load(gpkg_path, layer_name)
    #print(gdf.head())  # Display first few rows for verification
gdf


2024-12-12 12:30:21,363 - INFO - Loaded layer 'vaarweg_markering_drijvend_detailPoint' from ..\data\converted_datav2.gpkg
2024-12-12 12:30:21,364 - INFO - Loaded OGR Arrow stream
2024-12-12 12:30:21,366 - INFO - Schema retrieved: struct<fid: int64 not null, vaarwater: string, benam_cod: string, benaming: string, inbedrijf: string, x_rd: string, y_rd: string, obj_soort: string, iala_categ: string, n_wgs_gms: string, e_wgs_gms: string, n_wgs_gm: string, e_wgs_gm: string, obj_vorm_c: string, obj_vorm: string, obj_kleur_: string, obj_kleur: string, kleurpatr_: string, kleurpatr: string, v_tt_c: string, tt_toptek: string, tt_kleur_c: string, tt_kleur: string, tt_pat_c: string, tt_klr_pat: string, sign_kar_c: string, sign_kar: string, sign_gr_c: string, sign_groep: string, sign_perio: string, racon_code: string, licht_kl_c: string, licht_klr: string, opgeheven: string, x_wgs84: double, y_wgs84: double, vorm_kleur: string, s57_id: int32, geom: binary>
2024-12-12 12:30:21,366 - INFO - Field na

,fid,vaarwater,benam_cod,benaming,inbedrijf,x_rd,y_rd,obj_soort,iala_categ,n_wgs_gms,e_wgs_gms,n_wgs_gm,e_wgs_gm,obj_vorm_c,obj_vorm,obj_kleur_,obj_kleur,kleurpatr_,kleurpatr,v_tt_c,tt_toptek,tt_kleur_c,tt_kleur,tt_pat_c,tt_klr_pat,sign_kar_c,sign_kar,sign_gr_c,sign_groep,sign_perio,racon_code,licht_kl_c,licht_klr,opgeheven,x_wgs84,y_wgs84,vorm_kleur,s57_id,geometry
0,1,AARDAPPELENGAT,VW-A -0101,A 21,01-01-2005,"72384,1773","421540,5036",SK31 630,4,51.46.35.11,004.11.24.30,51.46.5852,004.11.4050,5,spar,4,Groen,#,Niet toegewezen,1,"Kegel, punt naar boven",4,Groen,#,Niet toegewezen,#,Niet toegewezen,#,Niet toegewezen,#,#,#,Niet toegewezen,#,4.190083,51.776419,spar Groen,1044,POINT (72384.362 421540.675)
1,2,AARDAPPELENGAT,VW-A -0102,A 19,01-01-2005,"71915,3833","421600,9262",SK31 630,4,51.46.36.81,004.10.59.79,51.46.6135,004.10.9965,5,spar,4,Groen,#,Niet toegewezen,#,Niet toegewezen,#,Niet toegewezen,#,Niet toegewezen,#,Niet toegewezen,#,Niet toegewezen,#,#,#,Niet toegewezen,#,4.183275,51.776892,spar Groen,1042,POINT (71915.403 421600.961)
2,3,AARDAPPELENGAT,VW-A -0103,A 17,01-01-2005,"71305,1921","421800,7938",SK31 630,4,51.46.42.95,004.10.27.79,51.46.7158,004.10.4632,5,spar,4,Groen,#,Niet toegewezen,#,Niet toegewezen,#,Niet toegewezen,#,Niet toegewezen,#,Niet toegewezen,#,Niet toegewezen,#,#,#,Niet toegewezen,#,4.174386,51.778597,spar Groen,1042,POINT (71305.172 421800.882)
3,4,AARDAPPELENGAT,VW-A -0104,A 18,01-01-2005,"71394,1015","421958,7968",SK32 630,4,51.46.48.11,004.10.32.29,51.46.8018,004.10.5382,5,spar,3,Rood,#,Niet toegewezen,#,Niet toegewezen,#,Niet toegewezen,#,Niet toegewezen,#,Niet toegewezen,#,Niet toegewezen,#,#,#,Niet toegewezen,#,4.175636,51.780031,spar Rood,1142,POINT (71394.083 421958.894)
4,5,AARDAPPELENGAT,VW-A -0105,A 15,01-01-2005,"70730,0620","421975,8322",SK31 630,4,51.46.48.30,004.09.57.64,51.46.8050,004.09.9607,5,spar,4,Groen,#,Niet toegewezen,#,Niet toegewezen,#,Niet toegewezen,#,Niet toegewezen,#,Niet toegewezen,#,Niet toegewezen,#,#,#,Niet toegewezen,#,4.166011,51.780083,spar Groen,1042,POINT (70730.037 421975.871)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9906,9907,ZWOLSCHEDIEP (RANDMEREN),VW-ZDR -0132,LZD-N,01-01-2005,"198103,9045","516715,7031",SK31 630,3,52.38.10.46,006.01.26.21,52.38.1743,006.01.4369,5,spar,6,Geel,#,Niet toegewezen,5,Cilinder,"3,1,3",Rood/wit/rood,1,Horizontaal,#,Niet toegewezen,#,Niet toegewezen,#,#,#,Niet toegewezen,#,6.023947,52.636239,spar Geel,5541,POINT (198104.013 516716)
9907,9908,ZWOLSCHEDIEP (RANDMEREN),VW-ZDR -0133,LZD-O,01-01-2005,"198161,8658","516706,7554",SK31 630,3,52.38.10.15,006.01.29.29,52.38.1692,006.01.4882,5,spar,6,Geel,#,Niet toegewezen,5,Cilinder,"3,1,3",Rood/wit/rood,1,Horizontaal,#,Niet toegewezen,#,Niet toegewezen,#,#,#,Niet toegewezen,#,6.024803,52.636153,spar Geel,5541,POINT (198162.013 516706.929)
9908,9909,ZWOLSCHEDIEP (RANDMEREN),VW-ZDR -0134,LZD-P,01-01-2005,"198219,9401","516697,8093",SK31 630,3,52.38.09.85,006.01.32.38,52.38.1641,006.01.5396,5,spar,6,Geel,#,Niet toegewezen,5,Cilinder,"3,1,3",Rood/wit/rood,1,Horizontaal,#,Niet toegewezen,#,Niet toegewezen,#,#,#,Niet toegewezen,#,6.025661,52.636069,spar Geel,5541,POINT (198220.199 516698.169)
9909,9910,ZWOLSCHEDIEP (RANDMEREN),VW-ZDR -0135,LZD-Q,01-01-2005,"198265,9245","516690,7971",SK31 630,3,52.38.09.61,006.01.34.82,52.38.1601,006.01.5803,5,spar,6,Geel,#,Niet toegewezen,5,Cilinder,"3,1,3",Rood/wit/rood,1,Horizontaal,#,Niet toegewezen,#,Niet toegewezen,#,#,#,Niet toegewezen,#,6.026339,52.636003,spar Geel,5541,POINT (198266.146 516691.157)


In [81]:
#foutgevoelig, met het oog uit pdf gehaald en bekeken in Qgis
#aanwezigheid en locatie boei is in Qgis gecheckt.
#vaarwater : [boeien] relaties, vaarwater benodigd omdat sommige boeiennamen dubbel voorkwamen (bijvoorbeeld K 2 in ijselmeer)
vaarwater_boeien = {"KRAMMER": ["K 2"],
               "KEETEN-MASTGAT-ZIJPE" : ["KT 42-ZG 1", 
                                         "KEETEN A",
                                         "KEETEN B",
                                         "KT 1-EV 2",
                                         "KT 2",
                                         "KT 10",
                                         "KT 13",
                                         "KT 14",
                                         "KT 18", 
                                         "KT 21", 
                                         "KT 24", 
                                         "KT 28", 
                                         "KT 31", 
                                         "KT 27-KR 2"],
               "ENGELSCHE VAARWATER": ["EV 6",
                                       "EV 7-O 4",
                                       "EV 10"
                                       ],
               "OOSTERSCHELDE": ["O 1",
                                 "O 11-Z 2",
                                 "O 15-SAS 2",
                                 "O 18",
                                 "O 22",
                                 "O 25-SVY 2",
                                 "O 28-D 15",
                                 "O 33"],
               "ZANDKREEK": ["Z 1"],       #start 4, niet onderdeel?
               "THOLENSCHE GAT":["TG 4",  #start 3
                                  "TG 3"], #start 3
               "PIETERMANSKREEK": ["PK 4",
                                   "PK 8"],
               "LODIJKSCHE GAT": ["LG 5-PK 2",
                                  "LG 8",
                                  "LG 14"],
               "KRABBENKREEK":["KR 1", "KR 7", "KR 8", "KR 15"]}
#aantal boeien in dict
lenb = 0
for i in vaarwater_boeien:
    lenb += len(vaarwater_boeien[i]) 
print(lenb)#38 features
#boeien selecteren uit gdf
gdf_8uren = gdf[gdf.apply(lambda row: row["benaming"] in vaarwater_boeien.get(row["vaarwater"], []), axis=1)]
#aantal boeien gdf
print(len(gdf_8uren)) #38 features

#checks
if not len(gdf_8uren) == lenb:
    print("oeps, wss typefout in dict")
if not len(gdf_8uren) == 38:
    print("oeps, niet aantal als in kaart.pdf")


38
38


In [83]:
gdf_8uren.explore(popup="benaming")


In [ ]:
import folium

# Reproject GeoDataFrame to WGS84 (EPSG:4326)
gdf_8uren_wgs84 = gdf_8uren.to_crs(epsg=4326)

# Create a base map centered on the data
m = folium.Map(location=[
    gdf_8uren_wgs84.geometry.y.mean(),
    gdf_8uren_wgs84.geometry.x.mean()
], zoom_start=12)

# Add both circle markers and labels
for _, row in gdf_8uren_wgs84.iterrows():
    # Add a red circle marker
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=6,  # Circle size
        color="red",  # Circle border color
        fill=True,
        fill_color="red",
        fill_opacity=0.8
    ).add_to(m)
    
    # Add a label displaced to the right of the marker
    folium.Marker(
        location=[row.geometry.y + 0.1, row.geometry.x + 0.001],  # Displacement for text
        icon=folium.DivIcon(
            html=f'<div style="font-size: 12px; font-weight: bold; color: black; text-align: left; white-space: nowrap;">{row["benaming"]}</div>'
        )
    ).add_to(m)

# Display the map
m
